# core

> Module containing the reading and conversion logic from the sl2 and sl3 files to csv and shape

We need to do several processing steps to go from the `sl2` or `sl3` data to a `csv` and `shape` we can use in our GIS software.

**From depth in meter to bottom height in mNAP**

To convert from depth measurement in meter to bottom height measurement in mNAP we need to know the height in mNAP from which the depth measurements were taken. Currently we add this height manually in the filename in cmNAP. Some examples:

- `2024-07-11_zuiderpark Hoogeveen2_+1075cmnap.sl2`
- `Sonar_2022-04-26_21.07.11beschrijving+0765cmNAP.sl3`

We extract this height from the filename.

The height of the Sonar boot is also stored in the `gps_altitude` column from the `sl2` and `sl3` files, but we haven't yet implemented the conversion from this height to mNAP height.

**Filter relevant facts**

We only need the facts that have the value "primary" in the column "survey".

**Transformation to the correct CRS**

The coördinates in the Sonar files are in crs WGS84 (epsg:4326), we need to convert those to the crs we use, which is RDN Amersfoort (epsg:28992).
We accomplish that by using the `geopandas` method `.set_crs` and `.to_crs`.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

## Importing modules

We will use [sonarlight](https://github.com/KennethTM/sonarlight) to read the measurements from the `sl2` or `sl3` files.

::: {.callout-note}
Previously we used [sslib](https://github.com/opensounder/python-sllib) to parse the sonar files. But this package latest commit was 4 years ago. This `sslib` package also has less stars and the `sonarlight` has some neat extra features, such as simple conversion to a Pandas dataframe.
:::

In [ ]:
#| export
import geopandas as gpd
import pandas as pd
import re
from pathlib import Path
from sonarlight import Sonar

## Load sl3 file for testing.

In [ ]:
sl3_f = Path("../test/Sonar_2022-04-26_21.07.11beschrijving+0765cmNAP.sl3")

In [ ]:
#| export
def read_sl(
    filepath: Path # The absolute location of the file to convert
    )->Sonar:
    return Sonar(str(filepath))

In [ ]:
sl3_d = read_sl(sl3_f)
sl3_d

Summary of SL3 file:

- Primary channel with 1320 frames
- Secondary channel with 1320 frames
- Downscan channel with 1320 frames
- Sidescan channel with 1319 frames

Start time: 2022-04-26 11:08:49.101999998
End time: 2022-04-26 11:10:19.315000057

File info: version 3, device 2, blocksize 3200, frame version 10

In [ ]:
type(sl3_d)

sonarlight.sonar_class.Sonar

In [ ]:
sl3_d.df.head()

,id,survey,datetime,x,y,longitude,latitude,min_range,max_range,water_depth,gps_speed,gps_heading,gps_altitude,bottom_index,frames
1570,174,primary,2022-04-26 11:08:49.101999998,674208,6913399,6.076888,52.748741,0.000000,36.576000,0.606861,0.127106,0.246756,-1.91,50,"[137, 137, 137, 137, 137, 129, 124, 119, 114, ..."
1573,174,secondary,2022-04-26 11:08:49.101999998,674208,6913399,6.076888,52.748741,0.000000,36.576000,0.606861,0.127106,0.246756,-1.91,50,"[137, 137, 137, 137, 137, 129, 124, 119, 114, ..."
1576,174,downscan,2022-04-26 11:08:49.239000082,674207,6913400,6.076879,52.748746,0.000000,21.945601,0.606861,0.127106,0.246756,-1.91,38,"[152, 152, 152, 152, 129, 143, 140, 137, 140, ..."
1577,352,sidescan,2022-04-26 11:08:49.240000010,674207,6913400,6.076879,52.748746,-39.989758,39.989758,0.606861,0.127106,0.246756,-1.91,21,"[42, 26, 41, 38, 43, 46, 43, 47, 50, 50, 53, 5..."
1579,175,primary,2022-04-26 11:08:49.249000072,674208,6913399,6.076888,52.748741,0.000000,3.992880,0.609836,0.122237,0.247439,-1.91,469,"[216, 216, 216, 216, 216, 216, 216, 216, 216, ..."


In [ ]:
sl3_df_sml = sl3_d.df[0:16]
sl3_df_sml

,id,survey,datetime,x,y,longitude,latitude,min_range,max_range,water_depth,gps_speed,gps_heading,gps_altitude,bottom_index,frames
1570,174,primary,2022-04-26 11:08:49.101999998,674208,6913399,6.076888,52.748741,0.000000,36.576000,0.606861,0.127106,0.246756,-1.91,50,"[137, 137, 137, 137, 137, 129, 124, 119, 114, ..."
1573,174,secondary,2022-04-26 11:08:49.101999998,674208,6913399,6.076888,52.748741,0.000000,36.576000,0.606861,0.127106,0.246756,-1.91,50,"[137, 137, 137, 137, 137, 129, 124, 119, 114, ..."
1576,174,downscan,2022-04-26 11:08:49.239000082,674207,6913400,6.076879,52.748746,0.000000,21.945601,0.606861,0.127106,0.246756,-1.91,38,"[152, 152, 152, 152, 129, 143, 140, 137, 140, ..."
1577,352,sidescan,2022-04-26 11:08:49.240000010,674207,6913400,6.076879,52.748746,-39.989758,39.989758,0.606861,0.127106,0.246756,-1.91,21,"[42, 26, 41, 38, 43, 46, 43, 47, 50, 50, 53, 5..."
1579,175,primary,2022-04-26 11:08:49.249000072,674208,6913399,6.076888,52.748741,0.000000,3.992880,0.609836,0.122237,0.247439,-1.91,469,"[216, 216, 216, 216, 216, 216, 216, 216, 216, ..."
1582,175,secondary,2022-04-26 11:08:49.249000072,674208,6913399,6.076888,52.748741,0.000000,3.992880,0.609836,0.122237,0.247439,-1.91,469,"[216, 216, 216, 216, 216, 216, 216, 216, 216, ..."
1585,175,downscan,2022-04-26 11:08:49.309000015,674207,6913400,6.076879,52.748746,0.000000,2.407920,0.609836,0.122237,0.247439,-1.91,354,"[152, 152, 152, 152, 152, 152, 152, 152, 152, ..."
1586,354,sidescan,2022-04-26 11:08:49.311000109,674207,6913400,6.076879,52.748746,-2.987040,2.987040,0.609836,0.122237,0.247439,-1.91,285,"[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 1..."
1587,176,primary,2022-04-26 11:08:49.290999889,674208,6913399,6.076888,52.748741,0.000000,3.992880,0.609836,0.118954,0.248092,-1.91,469,"[216, 216, 216, 216, 216, 216, 216, 216, 216, ..."
1590,176,secondary,2022-04-26 11:08:49.290999889,674208,6913399,6.076888,52.748741,0.000000,3.992880,0.609836,0.118954,0.248092,-1.91,469,"[216, 216, 216, 216, 216, 216, 216, 216, 216, ..."


## Notes to use `gps_altitude` column from `sl3`-file instead of extrachting the height of the measurement instrument from the filename

The `sl3_df_sml` dataframe also has a column `gps_altitude`. Can you explain what this probably is. Given that longitude and latitude are given in the WGS84 coordinate system?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The `gps_altitude` column is likely the **altitude (elevation) above the WGS84 ellipsoid**, measured in meters. 

In the WGS84 coordinate system, GPS devices typically provide three coordinates:
- Longitude and latitude (horizontal position)
- Altitude/elevation (vertical position)

This GPS altitude is usually the height above the WGS84 reference ellipsoid (a mathematical model of Earth's shape), not the height above sea level or above a local datum like NAP.

Looking at your data, the `gps_altitude` values are around 12.19 meters. This would be the height of the GPS antenna (on your sonar boat) above the WGS84 ellipsoid at that location.

Is this different from the height information you're extracting from the filename (like "+1075cmnap")? What do you think the relationship might be between these two height measurements?

Great. How can I transform this `gps_altitude` to mNAP as used in the RDnew Amersfoort system (epsg:28992)?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

However, there's an important distinction to make here:

**GPS altitude and coordinate system transformations work differently than horizontal coordinates.**

When you transform longitude/latitude from WGS84 (epsg:4326) to RD New (epsg:28992) using `.to_crs()`, it handles the horizontal position transformation well. But the vertical component (altitude) is trickier because:

1. GPS altitude is height above the WGS84 ellipsoid
2. NAP (Normaal Amsterdams Peil) is a height above mean sea level
3. These two reference surfaces don't have a simple constant offset - the difference varies by location

To properly convert GPS altitude to mNAP, you typically need a **geoid model** or a transformation grid specific to the Netherlands (like NLGEO2018 or similar). This accounts for the local variations between the ellipsoid and the geoid (mean sea level).

Do you have access to such a transformation tool, or is there a reason you're interested in converting the GPS altitude? I notice you're already extracting the NAP height from the filename - is that serving a different purpose?

It would be good if we could skip the NAP height from the filename. Since that is a manual proces. Which may give errors or be forgotten when the file is uploaded.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

That makes sense - automating this would definitely reduce errors and make the workflow more robust!

To convert GPS altitude (WGS84 ellipsoid height) to mNAP, you'll need to apply a geoid correction. In the Netherlands, the standard approach is to use the **RDNAPTRANS** transformation, which includes both horizontal (RD) and vertical (NAP) transformations.

Python libraries that can handle this include:
- **pyproj** - can handle vertical transformations with the right geoid model
- **PROJ** (the underlying library) - supports compound CRS transformations

The key is to use a compound coordinate reference system that includes both horizontal and vertical components. For the Netherlands, you'd want something like EPSG:7415 (RD New + NAP height).

Have you used `pyproj` before, or would you like to explore how to set up this transformation? It will involve a few steps beyond the simple `.to_crs()` you're currently using.

## Convert Pandas DataFrame to GeoDataFrame

How to convert the pandas dataframe `sl2df_sml` to a geodataframe? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To convert your pandas DataFrame `sl2df_sml` to a GeoDataFrame, you need to create geometry objects from your coordinate columns. Since you have `longitude` and `latitude` columns, you can use those to create Point geometries.

Here's how you'd approach it:

```python
gdf = geopandas.GeoDataFrame(sl2df_sml, geometry=geopandas.points_from_xy(sl2df_sml.longitude, sl2df_sml.latitude))
```

This creates Point geometries from your longitude/latitude columns. However, you'll also want to set the coordinate reference system (CRS) since these are WGS84 coordinates. Would you like to try writing that part yourself, or would you like a hint about which method to use?

In [ ]:
gdf = gpd.GeoDataFrame(sl3_df_sml, geometry=gpd.points_from_xy(sl3_df_sml.longitude, sl3_df_sml.latitude))

In [ ]:
gdf = gdf.set_crs(epsg=4326)

In [ ]:
gdf.crs

In [ ]:
gdf = gdf.to_crs(epsg=28992)

In [ ]:
gdf.crs

::: {.callout-tip collapse="true"}
## 🤖 AI regex chat
I want to build a regex to extract the height in all of the following cases:

`measurements_-720cmnap.sl2`
`measurements_+720cmNap.sl3`
`measurements_-1720cmnap.sl3`
`measurements-1720cmnap.sl3`
`measurements+20cmNAP.sl2`
`measurements+20CMNAP.sl2`

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! You want to extract the height value (including the sign) from those filenames. Let's break down what patterns you need to match:

- Either a leading `+` or `-` sign
- One or more digits
- Followed by `cmnap` (case-insensitive)

You only need the number with its sign.

To extract this pattern with regex, you'll need to use Python's `re` module. Here's the approach:

**Pattern components:**
- `[+-]` matches either + or -
- `\d+` matches one or more digits
- `(?=cmnap)` is a lookahead that checks for "cmnap" (case-insensitive) without including it in the match

You'll want to use `re.search()` to find the pattern in the filename string, and use the `re.IGNORECASE` flag to handle different capitalizations.

Would you like to try writing the regex pattern yourself, or would you like me to show you an example?

:::

In [ ]:
rgx = r"[+-]\d+(?=cmnap)"

In [ ]:
int(re.search(rgx, str(sl2_f), flags=re.IGNORECASE)[0])

In [ ]:
#| export
def extract_height(
    sl_filepath: Path, # The absolute location of the file to convert
    re_ptrn: str=r"[+-]?\d+(?=cmnap)"
    )->int: # The height of the measurement station in cm above NAP
    "Extract height from the filename in cmNAP"
    if 'cmnap' not in str(sl_filepath).lower():
        raise ValueError("The filename must contain the height of the Sonar boot at time of measurement in 'cmNAP' at the end of the filename (e.g. 'example_description_+1050cmNAP.sl2')")
    return int(re.search(re_ptrn, str(sl_filepath), flags=re.IGNORECASE)[0])

Test function `extract_height`

In [ ]:
(extract_height(Path("/some/where/afen22e34_1823cmNAP.sl2")),
extract_height(Path("/some/where/afen22e34-1823cmNAP.sl2")),
extract_height(Path("/some/where/afen22e34br_-1823cmNAP.sl2")))

In [ ]:
def slx2gdf(
    sl_filepath: Path, # The absolute location of the file to convert
    msrmnt_height: int, # Height of the measurement instrument at time of taking the measurements
    to_crs: str = "epsg:28992", # epsg code of crs to transform the coördinates to
    )->gpd:
    "Convert a sl2 or sl3 file to a GeoDataFrame with the given crs."
    s = Sonar(str(sl_filepath))
    df = s.df
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    gdf = gdf.set_crs(epsg=4326)
    return gdf.to_crs(to_crs)

Test function `slx2gdf`

In [ ]:
sl3_gdf = slx2gdf(sl3_f, 1823)

In [ ]:
"water_depth" in gdf.columns

In [ ]:
sl3_gdf.head()

## Building filter on GeoDataFrame

In [ ]:
sl3_gdf_pr = sl3_gdf[sl3_gdf["survey"]=="primary"]
sl3_gdf_pr.head()

,id,survey,datetime,x,y,longitude,latitude,min_range,max_range,water_depth,gps_speed,gps_heading,gps_altitude,bottom_index,frames,geometry
1570,174,primary,2022-04-26 11:08:49.101999998,674208,6913399,6.076888,52.748741,0.0,36.57600,0.606861,0.127106,0.246756,-1.91,50,"[137, 137, 137, 137, 137, 129, 124, 119, 114, ...",POINT (201568.299 529266.871)
1579,175,primary,2022-04-26 11:08:49.249000072,674208,6913399,6.076888,52.748741,0.0,3.99288,0.609836,0.122237,0.247439,-1.91,469,"[216, 216, 216, 216, 216, 216, 216, 216, 216, ...",POINT (201568.299 529266.871)
1587,176,primary,2022-04-26 11:08:49.290999889,674208,6913399,6.076888,52.748741,0.0,3.99288,0.609836,0.118954,0.248092,-1.91,469,"[216, 216, 216, 216, 216, 216, 216, 216, 216, ...",POINT (201568.299 529266.871)
1596,177,primary,2022-04-26 11:08:49.378000021,674208,6913399,6.076888,52.748741,0.0,3.99288,0.612811,0.114456,0.248715,-1.91,471,"[216, 216, 216, 216, 216, 216, 216, 216, 216, ...",POINT (201568.299 529266.871)
1604,178,primary,2022-04-26 11:08:49.423000097,674208,6913399,6.076888,52.748741,0.0,3.99288,0.612811,0.107427,0.249878,-1.91,471,"[216, 216, 216, 216, 216, 216, 216, 216, 216, ...",POINT (201568.299 529266.871)


::: {.callout-tip collapse="true"}
Please explain the DataFrame warning.

```python
sl2_gdf_pr['bottom_height'] = 12.53/100 - sl2_gdf_pr['water_depth']
```

```text
/app/data/.local/lib/python3.12/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
```

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

This warning occurs because of how you created `sl2df_sml_pr`. When you filtered the dataframe with:

```python
sl2df_sml_pr = sl2df_sml[sl2df_sml["survey"]=="primary"]
```

Pandas doesn't know if this created a **copy** of the data or just a **view** (reference) to the original data. When you then try to add a new column to `sl2df_sml_pr`, pandas warns you that changes might unexpectedly affect the original `sl2df_sml` dataframe (or might not work at all).

**The solution:** Use `.copy()` to explicitly create a new dataframe:

```python
sl2df_sml_pr = sl2df_sml[sl2df_sml["survey"]=="primary"].copy()
```

Or use `.loc[]` for the assignment as the warning suggests. 

Would you like to try fixing this yourself, or do you have questions about why this happens?
:::

In [ ]:
sl3_gdf_pr = sl3_gdf[sl3_gdf["survey"]=="primary"].copy()

In [ ]:
sl3_gdf_pr['bottom_height'] = 12.53/100 - sl3_gdf_pr['water_depth']

In [ ]:
sl3_gdf_pr.head()

::: {.callout-tip collapse="true"}
## 🤖AI chat: `latitude` and `longitude`

❔ _How can I check the precision of the `latitude` and `longitude` columns?_

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To check the precision of the `latitude` and `longitude` columns, you can look at how many decimal places the values have. Here are a few approaches:

**Option 1:** Convert to string and check the length after the decimal point:
```python
sl2_gdf_pr['latitude'].astype(str).str.split('.').str[1].str.len()
```

**Option 2:** Calculate the smallest difference between consecutive values to see the resolution.

**Option 3:** Look at a few sample values directly to visually inspect the decimal places.

Which approach would you like to try first, or do you have a specific reason for checking the precision?

---
❔ _"Do I understand correctly that the visualisation from a Dataframe shows the complete numbers and doesn't truncate?"_

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Not quite - pandas DataFrames **do truncate** displayed values by default for readability, even though the full precision is stored internally.

You can check the actual precision by:

1. Using `.iloc[0]` to access a single value directly (this shows more precision than the table view)
2. Checking pandas display options with `pd.get_option('display.precision')`
3. Setting display precision higher with `pd.set_option('display.precision', 10)` (or whatever number you want)

Try accessing a single latitude value directly - does it show more decimal places than what you see in the table display?Not quite - pandas DataFrames **do** truncate displayed values by default! When you see a value like `52.71157` in the display, there might actually be more decimal places that aren't being shown.

To see the full precision of the values, you can:

1. Use `pd.set_option('display.precision', 20)` to show more decimal places in the display
2. Access individual values directly with `.iloc[]` or `.loc[]` and print them
3. Check the actual dtype of the column - if it's `float64`, it can store about 15-17 significant decimal digits

Would you like to try one of these methods to see the full precision of your latitude/longitude values?
:::

In [ ]:
sl3_gdf_pr['longitude'].iloc[0]

::: {.callout-tip collapse="true"}
## 🤖 AI Chat: Geometry column

❔ _How can I check if the precision of the `geometry` column is the same?_

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! The `geometry` column contains Point objects (not just numbers), so you need to extract the coordinates from the geometry first.

You can access the x and y coordinates of a Point geometry using `.x` and `.y` attributes:

```python
sl2_gdf_pr['geometry'].iloc[0].x
sl2_gdf_pr['geometry'].iloc[0].y
```

This will show you the precision of the transformed coordinates (in RD New/EPSG:28992). Try it and compare - do they have similar precision to the original longitude/latitude values?
:::

In [ ]:
sl3_gdf_pr['geometry'].iloc[0].x
sl3_gdf_pr['geometry'].iloc[0].y

::: {.callout-tip collapse="true"}
## 🤖 AI chat: Check meaning of `bottom_index` column

❔ _I think that all measurements with the same "bottom_index" also have the same longitude and latitude and water_depth. How can I check that assumption?_

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good hypothesis! To check this, you could group by `bottom_index` and see if the values within each group are all the same.

One approach is to use `.groupby()` with `.nunique()` (number of unique values). If a column has only 1 unique value per group, then all values in that group are identical.

Try something like:

```python
sl2df_sml_pr.groupby('bottom_index')[['longitude', 'latitude', 'water_depth']].nunique()
```

This will show you how many unique values exist for each column within each `bottom_index` group. What do you expect to see if your assumption is correct?
:::

In [ ]:
sl3_gdf[sl3_gdf["survey"]=="primary"].groupby('bottom_index')[['longitude', 'latitude', 'water_depth']].nunique()

,longitude,latitude,water_depth
bottom_index,,,
50,1,1,1
328,1,2,1
330,2,2,1
332,1,1,1
333,2,2,2
...,...,...,...
1001,1,1,1
1005,2,1,1
1007,1,1,1


In [ ]:
sl3_d.df[sl3_d.df["bottom_index"]==475]

,id,survey,datetime,x,y,longitude,latitude,min_range,max_range,water_depth,gps_speed,gps_heading,gps_altitude,bottom_index,frames
9536,1111,primary,2022-04-26 11:09:52.848999977,674198,6913407,6.076798,52.748784,0.0,3.99288,0.617507,0.162986,4.224975,-5.5,475,"[214, 214, 214, 214, 214, 214, 214, 214, 214, ..."
9539,1111,secondary,2022-04-26 11:09:52.848999977,674198,6913407,6.076798,52.748784,0.0,3.99288,0.617507,0.162986,4.224975,-5.5,475,"[214, 214, 214, 214, 214, 214, 214, 214, 214, ..."
9544,1112,primary,2022-04-26 11:09:52.894999981,674198,6913407,6.076798,52.748784,0.0,3.99288,0.617507,0.164213,4.223330,-5.5,475,"[214, 214, 214, 214, 214, 214, 214, 214, 214, ..."
9547,1112,secondary,2022-04-26 11:09:52.894999981,674198,6913407,6.076798,52.748784,0.0,3.99288,0.617507,0.164213,4.223330,-5.5,475,"[214, 214, 214, 214, 214, 214, 214, 214, 214, ..."


::: {.callout-tip collapse="true"}
## 🤖 AI chat: Further investigate `bottom_index`

❔ _It is mostly true that the same bottom_index is the same location and depth. But not always. So we must filter the resulting table on same locations. Could we use something like `.unique` to only keep those points that have a unique `longitude`, `latitude` combination?_

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes, good thinking! You can use `.drop_duplicates()` to keep only unique combinations of longitude and latitude.

The method would look like:

```python
df_unique = df.drop_duplicates(subset=['longitude', 'latitude'])
```

This keeps the first occurrence of each unique longitude/latitude pair and removes the rest.

Would you like to try applying this to your filtered primary survey data? Or do you have questions about how `drop_duplicates()` works?
:::

In [ ]:
def clean_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame from sl2gdf
    msrmnt_height: int, # Height of measurement instrument in cm above NAP
    ) -> gpd.GeoDataFrame: # Cleaned GeoDataFrame with bottom_height column
    "Filter primary survey data, remove duplicates, and calculate bottom height in mNAP"
    gdf_primary = gdf[gdf["survey"] == "primary"].copy()
    gdf_unique = gdf_primary.drop_duplicates(subset=['longitude', 'latitude', 'water_depth'])
    gdf_unique.loc[:, 'bottom_height'] = msrmnt_height / 100 - gdf_unique['water_depth']
    return gdf_unique

In [ ]:
cln_gdf = clean_gdf(sl2_gdf, 1823)

NameError: name 'clean_gdf' is not defined

Why do I get this warning in the function `clean_gdf`?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The warning occurs because `.drop_duplicates()` might return a view rather than a copy, and then you're trying to modify it.

The fix is to add `.copy()` after `.drop_duplicates()`:

```python
gdf_unique = gdf_primary.drop_duplicates(subset=['longitude', 'latitude', 'water_depth']).copy()
```

This ensures `gdf_unique` is definitely a new dataframe, not a view. Would you like to update the function with this change?

Is this really the best way? We now create a bunch of copies in memory, just to clean the dataframe.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're absolutely right to question this! Creating multiple copies is inefficient, especially with large datasets.

A better approach would be to chain the operations and only make one copy at the end:

```python
gdf_clean = (gdf[gdf["survey"] == "primary"]
             .drop_duplicates(subset=['longitude', 'latitude', 'water_depth'])
             .copy())
gdf_clean['bottom_height'] = msrmnt_height / 100 - gdf_clean['water_depth']
```

Or even more efficiently, you could use `.assign()` to add the column without triggering the warning:

```python
return (gdf[gdf["survey"] == "primary"]
        .drop_duplicates(subset=['longitude', 'latitude', 'water_depth'])
        .assign(bottom_height=lambda x: msrmnt_height / 100 - x['water_depth']))
```

This creates only one copy and avoids the warning entirely. Would you like to try refactoring your function with one of these approaches?

There also is another issue. Sometimes we do have the same location (e.g. 'longitude', 'latitude') but different heights. This is probably due to the fact that the depth meter creates a pulse almost every second, but the location is rounded off to respectively 6 and 5 numbers after the diggit. So I actually want to drop the long-lat duplicates and take the mean of the merged 'water_depth'.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good observation! So you want to group by longitude and latitude, then take the mean of `water_depth` for each unique location.

You can use `.groupby()` with `.agg()` to accomplish this. The tricky part is handling the geometry column and deciding what to do with the other columns (like datetime, gps_speed, etc.).

For the geometry, you could keep the first point since they're all at the same location. For other columns, you'll need to decide: keep first? mean? something else?

Would you like to try writing the groupby operation yourself, or would you like me to show you an example of how to structure it?

In [ ]:
import pandas as pd

In [ ]:
gdf_grpd = gdf.groupby(['longitude', 'latitude'], as_index=False).agg(
    mean_depth=pd.NamedAgg(column="water_depth", aggfunc="mean"),
    min_depth=pd.NamedAgg(column="water_depth", aggfunc="min"),
    max_depth=pd.NamedAgg(column="water_depth", aggfunc="max"),
    geometery=pd.NamedAgg(column="geometry", aggfunc="first"),
    datetime=pd.NamedAgg(column="datetime", aggfunc="mean")
)


In [ ]:
gdf_grpd.head()

In [ ]:
def clean_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame from sl2gdf
    msrmnt_height: int, # Height of measurement instrument in cm above NAP
    ) -> gpd.GeoDataFrame: # Cleaned GeoDataFrame with bottom_height column
    "Filter primary survey data, remove duplicates, and calculate bottom height in mNAP"
    return (gdf[gdf["survey"]=="primary"]
            .drop_duplicates(subset=['longitude', 'latitude', 'water_depth'])
            .assign(bottom_height=lambda x: msrmnt_height / 100 - x['water_depth']))

In [ ]:
def clean_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame from sl2gdf
    msrmnt_height: int, # Height of measurement instrument in cm above NAP
    ) -> gpd.GeoDataFrame: # Cleaned GeoDataFrame with bottom_height column
    "Filter primary survey data, remove duplicates, and calculate bottom height in mNAP"
    return (gdf[gdf["survey"]=="primary"]
            .groupby(['longitude', 'latitude'], as_index=False).agg(
                mean_depth=pd.NamedAgg(column="water_depth", aggfunc="mean"),
                min_depth=pd.NamedAgg(column="water_depth", aggfunc="min"),
                max_depth=pd.NamedAgg(column="water_depth", aggfunc="max"),
                geometry=pd.NamedAgg(column="geometry", aggfunc="first"),
                datetime=pd.NamedAgg(column="datetime", aggfunc="mean")
            )
            .assign(bottom_height=lambda x: msrmnt_height / 100 - x['mean_depth']))

When I use `clean_gdf` the returned object is a Pandas Dataframe, not a GeoDataFrame. This probably is the result of the `.groupby` method that uses the `agg` method. How can I solve this? I want a GeoDataFrame?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good catch! When you use `.groupby().agg()` on a GeoDataFrame, it returns a regular DataFrame, losing the GeoDataFrame properties.

To fix this, you need to convert it back to a GeoDataFrame after the aggregation. You can use:

```python
gpd.GeoDataFrame(result, geometry='geometry', crs=gdf.crs)
```

Where `result` is your aggregated dataframe. You'll need to make sure to preserve the original CRS from the input GeoDataFrame.

Would you like to try adding this conversion to your `clean_gdf` function?

In [ ]:
#| export
def clean_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame from sl2gdf
    msrmnt_height: int, # Height of measurement instrument in cm above NAP
    ) -> gpd.GeoDataFrame: # Cleaned GeoDataFrame with bottom_height column
    "Filter primary survey data, remove duplicates, and calculate bottom height in mNAP"
    df_cln = (gdf[gdf["survey"]=="primary"]
            .groupby(['longitude', 'latitude'], as_index=False).agg(
                mean_depth=pd.NamedAgg(column="water_depth", aggfunc="mean"),
                min_depth=pd.NamedAgg(column="water_depth", aggfunc="min"),
                max_depth=pd.NamedAgg(column="water_depth", aggfunc="max"),
                geometry=pd.NamedAgg(column="geometry", aggfunc="first"),
                datetime=pd.NamedAgg(column="datetime", aggfunc="mean")
            )
            .assign(bottom_height=lambda x: msrmnt_height / 100 - x['mean_depth']))
    return gpd.GeoDataFrame(df_cln, geometry='geometry', crs=gdf.crs)

In [ ]:
cln_gdf = clean_gdf(sl2_gdf, 1823)

In [ ]:
type(cln_gdf)

In [ ]:
cln_gdf.head()

In [ ]:
cln_gdf['geometry'].iloc[0].x

I also would like to check if the Dataframe that is created from the sl2 or sl2 file contains the needed columns in the needed datatype. Should I write a function to check that? Should I add the checks within an existing function? Should I use Pydantic?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question about validation! You have a few options:

1. **Separate validation function** - keeps concerns separated, reusable
2. **Add checks within existing functions** - catches issues early where they matter
3. **Use Pydantic** - more robust, but adds dependency and complexity

For your case, I'd suggest starting simple: add basic checks within your existing functions (like `sl2gdf` or `clean_gdf`) to verify the required columns exist. Something like:

```python
required_cols = ['longitude', 'latitude', 'water_depth', 'survey']
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"Missing required columns...")
```

Pydantic would be great if you're building a larger system with complex validation needs, but might be overkill here.

Which approach feels right for your use case? Are you expecting the sonar file format to be stable, or do you need robust validation?

In [ ]:
#| export
def slx2gdf(
    sl_filepath: Path, # The absolute location of the file to convert
    to_crs: str = "epsg:28992", # epsg code of crs to transform the coördinates to
    survey_fltr: str = "primary", # Filter measurement facts on survey value
    )->gpd:
    "Convert a sl2 or sl3 file to a GeoDataFrame with the given crs."
    s = Sonar(str(sl_filepath))
    df = s.df
    required_cols = ['longitude', 'latitude', 'water_depth', 'survey']
    if not all(col in df.columns for col in required_cols):
        raise KeyError(f"Missing one or more of the required columns in the converted sl2 or sl3 file.\nRequired columns are: 'longitude', 'latitude', 'water_depth' and 'survey'")
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    gdf = gdf.set_crs(epsg=4326)
    return gdf.to_crs(to_crs)

In [ ]:
gdf = slx2gdf(sl2_f)
gdf.head()

In [ ]:
gdf.crs

In [ ]:
gdf['geometry'].iloc[0].x

I want to export a  GeoDataframe to different filetypes. See `export_gdf` function. How do I do that?

def export_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame to be saved
    fn: str, # Filename of the GeoDataFrame without extension
    folder_out: Path, # Absolute path to folder where files can be saved
    esri_shp: bool=True, # Save GeoDataFrame to Esri shapefile?
    csv: bool=True, # Save GeoDataFrame to comma seperated file?
    geopckg: bool=True, # Save GeoDataFrame to geopackage?
    )->None:
    if esri_shp: pass
    if csv: pass
    if geopckg: pass

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To export a GeoDataFrame to different file formats, you can use geopandas' built-in methods. Here's how to implement each:

**Shapefile:**
```python
gdf.to_file(folder_out / "filename.shp")
```

**CSV:**
```python
gdf.to_csv(folder_out / "filename.csv", index=False)
```

**GeoPackage:**
```python
gdf.to_file(folder_out / "filename.gpkg", driver="GPKG")
```

For the CSV export, you might want to drop the geometry column first (since CSV can't store geometry) or convert it to WKT format. What would you prefer - include geometry as WKT text, or just export the coordinate columns without the geometry object?

I don't want to drop the geometry column completely. I want to keep the x and y coordinates in a x and y column. How do I do that?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can extract the x and y coordinates from the geometry column before exporting to CSV. Here's how:

```python
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y
```

Then when you export to CSV, you can drop just the geometry column:

```python
gdf.drop(columns='geometry').to_csv(folder_out / f"{fn}.csv", index=False)
```

Would you like to add this to your `export_gdf` function?

In [ ]:
#| export
def export_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame to be saved
    fn: str, # Filename of the GeoDataFrame without extension
    folder_out: Path, # Absolute path to folder where files can be saved
    esri_shp: bool=True, # Save GeoDataFrame to Esri shapefile?
    csv: bool=True, # Save GeoDataFrame to comma seperated file?
    geopckg: bool=True, # Save GeoDataFrame to geopackage?
    )->None:
    if esri_shp: gdf.to_file(folder_out / f"{fn}.shp")
    if geopckg: gdf.to_file(folder_out / f"{fn}.gpkg", driver="GPKG")
    if csv:
        gdf['x'] = gdf.geometry.x
        gdf['y'] = gdf.geometry.y
        df = gdf.drop(columns=['geometry'])
        df.to_csv(folder_out / f"{fn}.csv", index=False)

In [ ]:
sl2_f.stem

In [ ]:
export_gdf(cln_gdf, sl2_f.stem, Path("../test/"))

Write a function that combines all the functions above. The function should accept a path to a sl2 or sl3 file and a path to store the resulting "shape", "csv" and "geopackage" files with the properly georeferenced GeoDataframe with the cleaned data and bottom_height column.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's how you can combine all the functions:

```python
def process_sonar_file(
    sl_filepath: Path, # Path to sl2 or sl3 file
    folder_out: Path, # Output folder for exported files
    to_crs: str = "epsg:28992" # Target CRS
    ) -> None:
    "Process sonar file: extract height, convert to GeoDataFrame, clean, and export"
    msrmnt_height = extract_height(sl_filepath)
    gdf = slx2gdf(sl_filepath, to_crs=to_crs)
    gdf_clean = clean_gdf(gdf, msrmnt_height)
    export_gdf(gdf_clean, sl_filepath.stem, folder_out)
```

This chains together: extracting the height from the filename, converting to GeoDataFrame, cleaning the data, and exporting to all three formats.

Does this capture what you need?

In [ ]:
#| export
def process_sonar_file(
    sl_filepath: Path, # Path to sl2 or sl3 file
    folder_out: Path, # Output folder for exported files
    to_crs: str = "epsg:28992" # Target CRS
    ) -> None:
    "Process sonar file: extract height, convert to GeoDataFrame, clean, and export"
    msrmnt_height = extract_height(sl_filepath)
    gdf = slx2gdf(sl_filepath, to_crs=to_crs)
    gdf_clean = clean_gdf(gdf, msrmnt_height)
    export_gdf(gdf_clean, sl_filepath.stem, folder_out)

In [ ]:
process_sonar_file(sl2_f, Path("../test"))

Can you explain what the difference is "under the hood" between `gdf.loc[:, 'y'] =` and just `gdf['y'] = `?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great question! Here's what's happening under the hood:

**`gdf['y'] = ...`** (chained indexing)
- Pandas first evaluates `gdf['y']`, which *might* return a view or a copy (pandas can't always tell)
- Then it tries to assign to that result
- Because pandas isn't sure if you're modifying the original or a copy, it warns you

**`gdf.loc[:, 'y'] = ...`** (explicit indexing)
- This is a single operation that tells pandas: "I want to modify the original DataFrame at all rows (`:`) and column `'y'`"
- Pandas knows you intend to modify the original, so no ambiguity, no warning

The `.loc[]` approach is more explicit about your intent to modify the DataFrame in place, which helps pandas optimize the operation and avoid unnecessary copies.

Does that clarify the difference?

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()